### <p style="font-family:JetBrains Mono; font-weight:bold; letter-spacing: 2px; color:#294B8E; font-size:140%; text-align:left; padding: 0px; border-bottom: 3px solid #294B8E">Toxic comments</p>

<a id="1.2"></a>
<div style="padding:20px;
             color:white;
             margin:10;
             font-size:170%;
             text-align:left;
             display:fill;
             border-radius:5px;
             background-color:#294B8E;
             overflow:hidden;
             font-weight:700"><span style='color:#CDA63A'>|</span> Content</div>

<div class="toc"><ul class="toc-item">
     <li><span><a href="#Loading-libraries" data-toc-modified-id="Loading-libraries-1"><span class="toc-item-num">1&nbsp;&nbsp;</ span>Loading libraries</a></span></li>
     <li><span><a href="#EDA" data-toc-modified-id="EDA-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>EDA</a></span></li>
    <li><span><a href="#Data-preprocessing" data-toc-modified-id="Data-preprocessing-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Data preprocessing</a></span></li>
    <li><span><a href="#Prepare-samples-for-training-models" data-toc-modified-id="Prepare-samples-for-training-models-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Prepare samples for training models</a></span></li>
    <li><span><a href="#Model-training" data-toc-modified-id="Model-training-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Model training</a></span></li>

<a id="Loading-libraries"></a>
<div style="padding:20px;
             color:white;
             margin:10;
             font-size:170%;
             text-align:left;
             display:fill;
             border-radius:5px;
             background-color:#294B8E;
             overflow:hidden;
             font-weight:700"><span style='color:#CDA63A'>|</span> Loading libraries</div>

In [1]:
import nltk
import string
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import precision_score, recall_score, precision_recall_curve, plot_precision_recall_curve

<a id="EDA"></a>
<div style="padding:20px;
             color:white;
             margin:10;
             font-size:170%;
             text-align:left;
             display:fill;
             border-radius:5px;
             background-color:#294B8E;
             overflow:hidden;
             font-weight:700"><span style='color:#CDA63A'>|</span> EDA</div>

In [2]:
data = pd.read_csv('labeled.csv')

In [3]:
data.head(5)

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0


In [4]:
#dataset size
data.shape

(14412, 2)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14412 entries, 0 to 14411
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   comment  14412 non-null  object 
 1   toxic    14412 non-null  float64
dtypes: float64(1), object(1)
memory usage: 225.3+ KB


<a id="Data-preprocessing"></a>
<div style="padding:20px;
            color:white;
            margin:10;
            font-size:170%;
            text-align:left;
            display:fill;
            border-radius:5px;
            background-color:#294B8E;
            overflow:hidden;
            font-weight:700"><span style='color:#CDA63A'>|</span> Data preprocessing</div>

In [6]:
data['toxic'] = data['toxic'].apply(int)

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14412 entries, 0 to 14411
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   comment  14412 non-null  object
 1   toxic    14412 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 225.3+ KB


In [8]:
#data distribution
data['toxic'].value_counts() 

0    9586
1    4826
Name: toxic, dtype: int64

In [9]:
#look at the toxic comments
for i in data[data['toxic'] == 1]['comment'].head(5):
    print(i)

Верблюдов-то за что? Дебилы, бл...

Хохлы, это отдушина затюканого россиянина, мол, вон, а у хохлов еще хуже. Если бы хохлов не было, кисель их бы придумал.

Собаке - собачья смерть

Страницу обнови, дебил. Это тоже не оскорбление, а доказанный факт - не-дебил про себя во множественном числе писать не будет. Или мы в тебя верим - это ты и твои воображаемые друзья?

тебя не убедил 6-страничный пдф в том, что Скрипалей отравила Россия? Анализировать и думать пытаешься? Ватник что ли?)



In [10]:
#look at non-toxic comments
for i in data[data['toxic'] == 0]['comment'].head(5):
    print(i)

В шапке были ссылки на инфу по текущему фильму марвел. Эти ссылки были заменены на фразу Репортим брипидора, игнорируем его посты. Если этого недостаточно, чтобы понять, что модератор абсолютный неадекват, и его нужно лишить полномочий, тогда эта борда пробивает абсолютное дно по неадекватности.

Почитайте посты у этого автора,может найдете что нибудь полезное. Надеюсь помог) https: pikabu.ru story obyichnyie budni dezsluzhbyi 4932098

Про графику было обидно) я так то проходил все серии гта со второй части по пятую, кроме гта 4. И мне не мешала графика ни в одной из частей. На компе у меня было куча видеокарт. Начиная с 32мб RIVA TNT и заканчивая 2Гб 560Ti на которой я спокойно играю который год в танки, гта5, ведьмака3 купил на распродаже и начал проходить. Да, не на ультрах. С пониженными текстурами. И не мешает. Я не понимаю дрочева на графике, требовать графику уровня плойки 4 минимум. Мне надо чтобы глаза не резало, только и всего. По поводу управления, мне не хватает переходника

<a id="Prepare-samples-for-training-models"></a>
<div style="padding:20px;
            color:white;
            margin:10;
            font-size:170%;
            text-align:left;
            display:fill;
            border-radius:5px;
            background-color:#294B8E;
            overflow:hidden;
            font-weight:700"><span style='color:#CDA63A'>|</span> Prepare samples for training models</div>

In [11]:
train, test = train_test_split(data, test_size=500)

In [12]:
test.shape

(500, 2)

In [13]:
train['toxic'].value_counts()

0    9264
1    4648
Name: toxic, dtype: int64

In [14]:
test['toxic'].value_counts()

0    322
1    178
Name: toxic, dtype: int64

In [15]:
sentence_example = data.iloc[1]["comment"]

In [16]:
snowball = SnowballStemmer(language='russian')
russian_stop_words = stopwords.words('russian')
 
def tokenize_sentence(sentence:str, remove_stop_words: bool = True):
    tokens = word_tokenize(sentence, language = 'russian')
    tokens = [i for i in tokens if i not in string.punctuation]
    if remove_stop_words: 
        tokens = [i for i in tokens if i not in russian_stop_words]
    tokens = [snowball.stem(i) for i in tokens]
    return tokens

In [17]:
model_pipeline = Pipeline([("vectorizer", TfidfVectorizer(tokenizer=lambda x: tokenize_sentence(x, remove_stop_words=True))),
                           ("model", LogisticRegression(random_state=0))])

In [ ]:
model_pipeline.fit(train['comment'], train['toxic'])

In [ ]:
model_pipeline.predict(train['comment'])

In [ ]:
precision_score(y_true=test["toxic"], y_pred=model_pipeline.predict(test["comment"]) )

In [ ]:
recall_score(y_true=test["toxic"], y_pred=model_pipeline.predict(test["comment"]) )

In [ ]:
prec, rec, threshold = precision_recall_curve(y_true=test["toxic"], probas_pred=model_pipeline.predict_proba(test["comment"])[:, 1])

In [ ]:
plot_precision_recall_curve(estimator=model_pipeline, X = test["comment"], y = test["toxic"])

In [ ]:
np.where(prec > 0.95)

In [ ]:
threshold[313]

In [ ]:
precision_score(y_true=test["toxic"], y_pred=model_pipeline.predict_proba(test["comment"])[:, 1] > threshold[313])

In [ ]:
recall_score(y_true=test["toxic"], y_pred=model_pipeline.predict_proba(test["comment"])[:, 1] > threshold[313])

In [ ]:
grid_pipeline = Pipeline([("vectorizer", TfidfVectorizer(tokenizer=lambda x: tokenize_sentence(x, remove_stop_words=True))),
                           ("model", GridSearchCV(LogisticRegression(random_state=0),
                                                 param_grid={'C': [0.1, 1, 10.]},
                                                 cv=3,
                                                 verbose=4))])


In [ ]:
grid_pipeline.fit(train["comment"], train["toxic"])

In [ ]:
model_pipeline_c_10 = Pipeline([("vectorizer", TfidfVectorizer(tokenizer=lambda x: tokenize_sentence(x, remove_stop_words=True))),
                           ("model", LogisticRegression(random_state=0, C=10.))])

In [ ]:
model_pipeline_c_10.fit(train["comment"], train["toxic"])

In [ ]:
prec_c_10, rec_c_10, threshold_c_10 = precision_recall_curve(y_true=test["toxic"], probas_pred=model_pipeline_c_10.predict_proba(test["comment"])[:, 1])

In [ ]:
np.where(prec_c_10 > 0.95)

In [ ]:
precision_score(y_true=test["toxic"], y_pred=model_pipeline_c_10.predict_proba(test["comment"])[:, 1] > threshold[316])

In [ ]:
recall_score(y_true=test["toxic"], y_pred=model_pipeline_c_10.predict_proba(test["comment"])[:, 1] > threshold[316])